In [ ]:
# export
from local.imports import *
from local.notebook.core import *
import nbformat,inspect
from nbformat.sign import NotebookNotary

In [ ]:
# default_exp notebook.export
# default_cls_lvl 3

# Converting notebooks to modules

> The functions that transform the dev notebooks in the fastai library

- author: "Sylvain Gugger"

## Reading a notebook

### What's a notebook?

A jupyter notebook is a json file behind the scenes. We can just read it with the json module, which will return a nested dictionary of dictionaries/lists of dictionaries, but there are some small differences between reading the json and using the tools from `nbformat` so we'll use this one.

In [ ]:
#export
def read_nb(fname):
    "Read the notebook in `fname`."
    with open(Path(fname),'r') as f: return nbformat.reads(f.read(), as_version=4)

`fname` can be a string or a pathlib object.

In [ ]:
test_nb = read_nb('91_notebook_export.ipynb')

The root has four keys: `cells` contains the cells of the notebook, `metadata` some stuff around the version of python used to execute the notebook, `nbformat` and `nbformat_minor` the version of nbformat. 

In [ ]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [ ]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Python 3',
  'language': 'python',
  'name': 'python3'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.3'}}

In [ ]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.2'

The cells key then contains a list of cells. Each one is a new dictionary that contains entries like the type (code or markdown), the source (what is written in the cell) and the output (for code cells).

In [ ]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {'hide_input': False},
 'outputs': [],
 'source': '# export\nfrom local.imports import *\nfrom local.notebook.core import *\nimport nbformat,inspect\nfrom nbformat.sign import NotebookNotary'}

### Finding patterns

In [ ]:
# export
def check_re(cell, pat, code_only=True):
    "Check if `cell` contains a line with regex `pat`"
    if code_only and cell['cell_type'] != 'code': return
    if isinstance(pat, str): pat = re.compile(pat, re.IGNORECASE | re.MULTILINE)
    return pat.search(cell['source'])

`pat` can be a string or a compiled regex, if `code_only=True`, ignores markdown cells.

In [ ]:
cell = test_nb['cells'][0].copy()
assert check_re(cell, '# export') is not None
assert check_re(cell, re.compile('# export')) is not None
assert check_re(cell, '# bla') is None
cell['cell_type'] = 'markdown'
assert check_re(cell, '# export') is None
assert check_re(cell, '# export', code_only=False) is not None

In [ ]:
# export
_re_blank_export = re.compile(r"""
# Matches any line with #export or #exports without any module name:
^         # beginning of line (since re.MULTILINE is passed)
\s*       # any number of whitespace
\#\s*     # # then any number of whitespace
exports?  # export or exports
\s*       # any number of whitespace
$         # end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
_re_mod_export = re.compile(r"""
# Matches any line with #export or #exports with a module name and catches it in group 1:
^         # beginning of line (since re.MULTILINE is passed)
\s*       # any number of whitespace
\#\s*     # # then any number of whitespace
exports?  # export or exports
\s*       # any number of whitespace
(\S+)     # catch a group with any non-whitespace chars
\s*       # any number of whitespace
$         # end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module."
    if check_re(cell, _re_blank_export):
        if default is None:
            print(f"This cell doesn't have an export destination and was ignored:\n{cell['source'][1]}")
        return default
    tst = check_re(cell, _re_mod_export)
    return os.path.sep.join(tst.groups()[0].split('.')) if tst else None

The cells to export are marked with an `#export` or `#exports` code, potentially with a module name where we want it exported. The default is given in a cell of the form `#default_exp bla` inside the notebook (usually at the top), though in this function, it needs the be passed (the final script will read the whole notebook to find it).

In [ ]:
cell = test_nb['cells'][0].copy()
assert is_export(cell, 'export') == 'export'
cell['source'] = "# exports" 
assert is_export(cell, 'export') == 'export'
cell['source'] = "# export mod" 
assert is_export(cell, 'export') == 'mod'
cell['source'] = "# export mod.file" 
assert is_export(cell, 'export') == 'mod/file'
cell['source'] = "# expt mod.file"
assert is_export(cell, 'export') is None

In [ ]:
# export
_re_default_exp = re.compile(r"""
# Matches any line with #default_exp with a module name and catches it in group 1:
^            # beginning of line (since re.MULTILINE is passed)
\s*          # any number of whitespace
\#\s*        # # then any number of whitespace
default_exp  # export or exports
\s*          # any number of whitespace
(\S+)        # catch a group with any non-whitespace chars
\s*          # any number of whitespace
$            # end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [ ]:
# export
def find_default_export(cells):
    "Find in `cells` the default export module."
    for cell in cells:
        tst = check_re(cell, _re_default_exp)
        if tst: return tst.groups()[0]

Stops at the first cell containing a `#default_exp` code and return the value behind. Returns `None` if there are no cell with that code.

In [ ]:
assert find_default_export(test_nb['cells']) == 'notebook.export'
assert find_default_export(test_nb['cells'][2:]) is None

### Exporting notebooks

We're now ready to export notebooks!

In [ ]:
# export
def _create_mod_file(fname, nb_path):
    "Create a module file for `fname`."
    fname.parent.mkdir(parents=True, exist_ok=True)
    with open(fname, 'w') as f:
        f.write(f"#AUTOGENERATED! DO NOT EDIT! File to edit: dev/{nb_path.name} (unless otherwise specified).")
        f.write('\n\n__all__ = []')

In [ ]:
#export
_re_patch_func = re.compile(r"""
# Catches any function decorated with @patch, its name in group 1 and the patched class in group 2
@patch       # At any place in the cell, something that begins with @patch
\s*def       # Any number of whitespace (including a new line probably) followed by def
\s+          # One whitespace or more
([^\(\s]*)   # Catch a group composed of anything but whitespace or an opening parenthesis (name of the function)
\s*\(        # Any number of whitespace followed by an opening parenthesis
[^:]*        # Any number of character different of : (the name of the first arg that is type-annotated)
:\s*         # A column followed by any number of whitespace
([^,\)\s]*)  # Catch a group composed of anything but a comma, a closing parenthesis or whitespace (name of the class)
\s*          # Any number of whitespace
(?:,|\))     # Non-catching group with either a comma or a closing parenthesis
""", re.VERBOSE)

In [ ]:
#hide
tst = _re_patch_func.search("""
@patch
def func(obj:Class)""")
assert tst.groups() == ("func", "Class")
tst = _re_patch_func.search("""
@patch
def func (obj:Class, a)""")
assert tst.groups() == ("func", "Class")

In [ ]:
#export
_re_class_func_def = re.compile(r"""
# Catches any 0-indented function or class definition with its name in group 1
^              # Beginning of a line (since re.MULTILINE is passed)
(?:def|class)  # Non-catching group for def or class
\s+            # One whitespace or more
([^\(\s]*)     # Catching group with any character except an opening parenthesis or a whitespace (name)
\s*            # Any number of whitespace
(?:\(|:)       # Non-catching group with either an opening parenthesis or a : (classes don't need ())
""", re.MULTILINE | re.VERBOSE)

In [ ]:
#hide
assert _re_class_func_def.search("class Class:").groups() == ('Class',)
assert _re_class_func_def.search("def func(a, b):").groups() == ('func',)

In [ ]:
#export
_re_obj_def = re.compile(r"""
# Catches any 0-indented object definition (bla = thing) with its name in group 1
^          # Beginning of a line (since re.MULTILINE is passed)
([^=\s]*)  # Catching group with any character except a whitespace or an equal sign
\s*=       # Any number of whitespace followed by an =
""", re.MULTILINE | re.VERBOSE)

In [ ]:
#hide
assert _re_obj_def.search("a = 1").groups() == ('a',)
assert _re_obj_def.search("a=1").groups() == ('a',)

In [ ]:
# export
def _not_private(n):
    for t in n.split('.'):
        if t.startswith('_'): return False
    return '\\' not in t and '^' not in t and '[' not in t

def export_names(code, func_only=False):
    "Find the names of the objects, functions or classes defined in `code` that are exported."
    #Format monkey-patches with @patch
    code = _re_patch_func.sub(r'def \2.\1() = ', code)
    names = _re_class_func_def.findall(code)
    if not func_only: names += _re_obj_def.findall(code)
    return [n for n in names if _not_private(n)]

This function only picks the zero-indented objects, functions or classes (we don't want the class methods for instance) and excludes private names (that begin with `_`). It only returns func and class names when `func_only=True`.

In [ ]:
assert export_names("def my_func(x):\n  pass\nclass MyClass():") == ["my_func", "MyClass"]
#Indented funcs are ignored (funcs inside a class)
assert export_names("  def my_func(x):\n  pass\nclass MyClass():") == ["MyClass"]
#Private funcs are ignored
assert export_names("def _my_func():\n  pass\nclass MyClass():") == ["MyClass"]
#trailing spaces
assert export_names("def my_func ():\n  pass\nclass MyClass():") == ["my_func", "MyClass"]
#class without parenthesis
assert export_names("def my_func ():\n  pass\nclass MyClass:") == ["my_func", "MyClass"]
#object and funcs
assert export_names("def my_func ():\n  pass\ndefault_bla=[]:") == ["my_func", "default_bla"]
assert export_names("def my_func ():\n  pass\ndefault_bla=[]:", func_only=True) == ["my_func"]
#Private objects are ignored
assert export_names("def my_func ():\n  pass\n_default_bla = []:") == ["my_func"]
#Objects with dots are privates if one part is private
assert export_names("def my_func ():\n  pass\ndefault.bla = []:") == ["my_func", "default.bla"]
assert export_names("def my_func ():\n  pass\ndefault._bla = []:") == ["my_func"]
#Monkey-path with @patch are properly renamed
assert export_names("@patch\ndef my_func(x:Class):\n  pass") == ["Class.my_func"]
assert export_names("@patch\ndef my_func(x:Class):\n  pass", func_only=True) == ["Class.my_func"]
assert export_names("some code\n@patch\ndef my_func(x:Class, y):\n  pass") == ["Class.my_func"]

In [ ]:
#export
_re_all_def   = re.compile(r"""
# Catches a cell with defines \_all\_ = [\*\*] and get that \*\* in group 1
^_all_   #  Beginning of line (since re.MULTILINE is passed)
\s*=\s*  #  Any number of whitespace, =, any number of whitespace
\[       #  Opening [
([^\n\]]*) #  Catching group with anything except a ] or newline
\]       #  Closing ]
""", re.MULTILINE | re.VERBOSE)

#Same with __all__
_re__all__def = re.compile(r'^__all__\s*=\s*\[([^\]]*)\]', re.MULTILINE)

In [ ]:
# export
def extra_add(code):
    "Catch adds to `__all__` required by a cell with `_all_=`"
    if _re_all_def.search(code):
        names = _re_all_def.search(code).groups()[0]
        names = re.sub('\s*,\s*', ',', names)
        names = names.replace('"', "'")
        code = _re_all_def.sub('', code)
        code = re.sub(r'([^\n]|^)\n*$', r'\1', code)
        return names.split(','),code
    return [],code

In [ ]:
assert extra_add('_all_ = ["func", "func1", "func2"]') == (["'func'", "'func1'", "'func2'"],'')
assert extra_add('_all_ = ["func",   "func1" , "func2"]') ==  (["'func'", "'func1'", "'func2'"],'')
assert extra_add("_all_ = ['func','func1', 'func2']\n") ==  (["'func'", "'func1'", "'func2'"],'')
assert extra_add('code\n\n_all_ = ["func", "func1", "func2"]') == (["'func'", "'func1'", "'func2'"],'code')

In [ ]:
#export
def _add2add(fname, names, line_width=120):
    if len(names) == 0: return
    with open(fname, 'r') as f: text = f.read()
    tw = TextWrapper(width=120, initial_indent='', subsequent_indent=' '*11, break_long_words=False)
    re_all = _re__all__def.search(text)
    start,end = re_all.start(),re_all.end()
    text_all = tw.wrap(f"{text[start:end-1]}{'' if text[end-2]=='[' else ', '}{', '.join(names)}]")
    with open(fname, 'w') as f: f.write(text[:start] + '\n'.join(text_all) + text[end:])

In [ ]:
fname = 'test_add.txt'
with open(fname, 'w') as f: f.write("Bla\n__all__ = [my_file, MyClas]\nBli")
_add2add(fname, ['new_function'])
with open(fname, 'r') as f: 
    assert f.read() == "Bla\n__all__ = [my_file, MyClas, new_function]\nBli"
_add2add(fname, [f'new_function{i}' for i in range(10)])
with open(fname, 'r') as f: 
    assert f.read() == """Bla
__all__ = [my_file, MyClas, new_function, new_function0, new_function1, new_function2, new_function3, new_function4,
           new_function5, new_function6, new_function7, new_function8, new_function9]
Bli"""
os.remove(fname)

In [ ]:
# export
def _relative_import(name, fname):
    mods = name.split('.')
    splits = str(fname).split(os.path.sep)
    if mods[0] not in splits: return name
    splits = splits[splits.index(mods[0]):]
    while splits[0] == mods[0]: splits,mods = splits[1:],mods[1:]
    return '.' * (len(splits)) + '.'.join(mods)

In [ ]:
assert _relative_import('local.core', Path('local')/'data.py') == '.core'
assert _relative_import('local.core', Path('local')/'vision'/'data.py') == '..core'
assert _relative_import('local.vision.transform', Path('local')/'vision'/'data.py') == '.transform'
assert _relative_import('local.notebook.core', Path('local')/'data'/'external.py') == '..notebook.core'

In [ ]:
#export
#Catches any from local.bla import something and catches local.bla in group 1, the imported thing(s) in group 2.
_re_import = re.compile(r'^\s*from (local.\S*) import (.*)$')

In [ ]:
# export
def _deal_import(code_lines, fname):
    pat = re.compile(r'from (local.\S*) import (\S*)$')
    lines = []
    def _replace(m):
        mod,obj = m.groups()
        return f"from {_relative_import(mod, fname)} import {obj}"
    for line in code_lines:
        line = re.sub('_'+'file_', '__'+'file__', line) #Need to break _file_ or that line will be treated
        lines.append(_re_import.sub(_replace,line))
    return lines

In [ ]:
#hide
lines = ["from local.core import *", "nothing to see", "from local.vision import bla1, bla2"]
assert _deal_import(lines, Path('local')/'data.py') == [
    "from .core import *", "nothing to see", "from .vision import bla1, bla2"
]

In [ ]:
#hide
#Tricking jupyter notebook to have a __file__ attribute. All _file_ will be replaced by __file__
_file_ = Path('local').absolute()/'notebook'/'export.py'

In [ ]:
#export
def _get_index():
    if not (Path(_file_).parent/'index.txt').exists(): return {}
    return json.load(open(Path(_file_).parent/'index.txt', 'r'))

def _save_index(index): 
    fname = Path(_file_).parent/'index.txt'
    fname.parent.mkdir(parents=True, exist_ok=True)
    json.dump(index, open(fname, 'w'), indent=2)
    
def _reset_index():
    if (Path(_file_).parent/'index.txt').exists():
        os.remove(Path(_file_).parent/'index.txt')

In [ ]:
#hide
ind,ind_bak = Path(_file_).parent/'index.txt',Path(_file_).parent/'index.bak'
if ind.exists(): shutil.move(ind, ind_bak)
assert _get_index() == {}
_save_index({'foo':'bar'})
assert _get_index() == {'foo':'bar'}
if ind_bak.exists(): shutil.move(ind_bak, ind)

In [ ]:
def _update_pkl(fname, cell):
    dic = pickle.load(open((Path.cwd()/'lib.pkl'), 'rb')) if (Path.cwd()/'lib.pkl').exists() else collections.defaultdict(list)
    dic[fname].append(cell)
    pickle.dump(dic, open((Path.cwd()/'lib.pkl'), 'wb'))

In [ ]:
#export 
def _notebook2script(fname, silent=False, to_pkl=False):
    "Finds cells starting with `#export` and puts them into a new module"
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    fname = Path(fname)
    nb = read_nb(fname)
    default = find_default_export(nb['cells'])
    if default is not None:
        default = os.path.sep.join(default.split('.'))
        if not to_pkl: _create_mod_file(Path.cwd()/'local'/f'{default}.py', fname)
    index = _get_index()
    exports = [is_export(c, default) for c in nb['cells']]
    cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]
    for (i,c,e) in cells:
        fname_out = Path.cwd()/'local'/f'{e}.py'
        orig = ('#C' if e==default else f'#Comes from {fname.name}, c') + 'ell\n'
        code = '\n\n' + orig + '\n'.join(_deal_import(c['source'].split('\n')[1:], fname_out))
        # remove trailing spaces
        names = export_names(code)
        extra,code = extra_add(code)
        _add2add(fname_out, [f"'{f}'" for f in names if '.' not in f] + extra)
        index.update({f: fname.name for f in names})
        code = re.sub(r' +$', '', code, flags=re.MULTILINE)
        if code != '\n\n' + orig[:-1]:
            if to_pkl: _update_pkl(fname_out, (i, fname, code))
            else:
                with open(fname_out, 'a') as f: f.write(code)
    _save_index(index)
    if not silent: print(f"Converted {fname}.")

In [ ]:
_notebook2script('92_notebook_showdoc.ipynb', to_pkl=True)

Converted 92_notebook_showdoc.ipynb.


In [ ]:
#export 
def _get_sorted_files(all_fs: Union[bool,str], up_to=None):
    "Return the list of files corresponding to `g` in the current dir."
    if (all_fs==True): ret = glob.glob('*.ipynb') # Checks both that is bool type and that is True
    else: ret = glob.glob(all_fs) if isinstance(g,str) else []
    if len(ret)==0: print('WARNING: No files found')
    ret = [f for f in ret if not f.startswith('_')]
    if up_to is not None: ret = [f for f in ret if str(f)<=str(up_to)]
    return sorted(ret)

In [ ]:
#export 
def notebook2script(fname=None, all_fs=None, up_to=None, silent=False, to_pkl=False):
    "Convert `fname` or all the notebook satisfying `all_fs`."
    # initial checks
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    assert fname or all_fs
    if all_fs: _reset_index()
    if (all_fs is None) and (up_to is not None): all_fs=True # Enable allFiles if upTo is present
    fnames = _get_sorted_files(all_fs, up_to=up_to) if all_fs else [fname]
    [_notebook2script(f, silent=silent, to_pkl=to_pkl) for f in fnames]

Finds cells starting with `#export` and puts them into the appropriate module.
* `fname`: the filename of one notebook to convert
* `all_fs`: `True` if you want to convert all notebook files in the folder or a glob expression
* `up_to`: converts all notebooks respecting the previous arg up to a certain number

Examples of use in console:
```
notebook2script                                 # Parse all files
notebook2script --fname 00_export.ipynb         # Parse 00_export.ipynb
notebook2script --all_fs=nb*                    # Parse all files starting with nb*
notebook2script --up_to=10                      # Parse all files with (name<='10')
notebook2script --all_fs=*_*.ipynb --up_to=10   # Parse all files with an '_' and (name<='10')
```

### Finding the way back to notebooks

In [ ]:
# export
def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    else:                              return str(obj).split('.')[-1]

In [ ]:
assert get_name(in_ipython) == 'in_ipython'
assert get_name(DocsTestClass.test) == 'test'
# assert get_name(Union[Tensor, float]) == 'Union'

In [ ]:
# export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [ ]:
assert qual_name(DocsTestClass) == 'DocsTestClass'
assert qual_name(DocsTestClass.test) == 'DocsTestClass.test'

In [ ]:
# export
def source_nb(func, is_name=None, return_all=False):
    "Return the name of the notebook where `func` was defined"
    is_name = is_name or isinstance(func, str)
    index = _get_index()
    name = func if is_name else qual_name(func)
    while len(name) > 0:
        if name in index: return (name,index[name]) if return_all else index[name]
        name = '.'.join(name.split('.')[:-1])

You can either pass an object or its name (by default `is_name` will look if `func` is a string or not, but you can override if there is some inconsistent behavior). 

If passed a method of a class, the function will return the notebook in which the largest part of the function was defined in case there is a monkey-matching that defines `class.method` in a different notebook than `class`. If `return_all=True`, the function will return a tuple with the name by which the function was found and the notebook.

In [ ]:
from local.data.pipeline import Transform
from local.test import test_fail

assert source_nb(test_fail) == '00_test.ipynb'
assert source_nb(Transform) == '02_data_transforms.ipynb'
assert source_nb(Transform.decode) == '02_data_transforms.ipynb'
#opt_call is in the core module but defined in 02
# from local.core import opt_call
# assert source_nb(opt_call) == '02_data_pipeline.ipynb' # TODO: find something else
assert source_nb(int) is None
#Added through a monkey-patch
assert source_nb('Path.ls') == '01_core.ipynb'

#Test with name
assert source_nb('DocsTestClass') == '90_notebook_core.ipynb'
assert source_nb('DocsTestClass.test') == '90_notebook_core.ipynb'

#Test return_all
assert source_nb(DocsTestClass, return_all=True) == ('DocsTestClass','90_notebook_core.ipynb')
assert source_nb(DocsTestClass.test, return_all=True) == ('DocsTestClass','90_notebook_core.ipynb')

## Reading the library

If someone decides to change a module instead of the notebooks, the following functions help update the notebooks accordingly.

In [ ]:
# export
_re_default_nb = re.compile(r'File to edit: dev/(\S+)\s+')
_re_cell = re.compile(r'^#Cell|^#Comes from\s+(\S+), cell')

In [ ]:
with open(Path.cwd()/'local'/'core.py') as f: code = f.read()

In [ ]:
_split(code)

[('01_core.ipynb',
  'from .test import *\nfrom .imports import *\nfrom .notebook.showdoc import show_doc'),
 ('01_core.ipynb', 'defaults = SimpleNamespace()'),
 ('01_core.ipynb',
  'class PrePostInitMeta(type):\n    "A metaclass that calls optional `__pre_init__` and `__post_init__` methods"\n    def __new__(cls, name, bases, dct):\n        x = super().__new__(cls, name, bases, dct)\n        def _pass(self, *args,**kwargs): pass\n        for o in (\'__init__\', \'__pre_init__\', \'__post_init__\'):\n            if not hasattr(x,o): setattr(x,o,_pass)\n        old_init = x.__init__\n\n        @functools.wraps(old_init)\n        def _init(self,*args,**kwargs):\n            self.__pre_init__()\n            old_init(self, *args,**kwargs)\n            self.__post_init__()\n        setattr(x, \'__init__\', _init)\n        return x'),
 ('01_core.ipynb',
  'class BaseObj(metaclass=PrePostInitMeta):\n    "Base class that provides `PrePostInitMeta` metaclass to subclasses"\n    pass'),
 ('01_co

In [ ]:
#export
def _split(code):
    lines = code.split('\n')
    default_nb = _re_default_nb.search(lines[0]).groups()[0]
    s,res = 1,[]
    while _re_cell.search(lines[s]) is None: s += 1
    e = s+1
    while e < len(lines):
        while e < len(lines) and _re_cell.search(lines[e]) is None: e += 1
        grps = _re_cell.search(lines[s]).groups()
        nb = grps[0] or default_nb
        content = lines[s+1:e]
        while len(content) > 1 and content[-1] == '': content = content[:-1]
        res.append((nb, '\n'.join(content)))
        s,e = e,e+1
    return res

In [ ]:
# export
def _relimport2name(name, mod_name):
    if mod_name.endswith('.py'): mod_name = mod_name[:-3]
    mods = mod_name.split(os.path.sep)
    mods = mods[mods.index('local'):]
    i = 0
    while name[i] == '.': i += 1
    return '.'.join(mods[:-i] + [name[i:]])

In [ ]:
assert _relimport2name('.core', 'local/data.py') == 'local.core'
assert _relimport2name('.core', 'home/sgugger/fastai_dev/dev/local/data.py') == 'local.core'
assert _relimport2name('..core', 'local/vision/data.py') == 'local.core'
assert _relimport2name('.transform', 'local/vision/data.py') == 'local.vision.transform'
assert _relimport2name('..notebook.core', 'local/data/external.py') == 'local.notebook.core'

In [ ]:
#export
#Catches any from .bla import something and catches local.bla in group 1, the imported thing(s) in group 2.
_re_loc_import = re.compile(r'^\s*from (\.\S*) import (.*)$')

In [ ]:
# export
def _deal_loc_import(code, fname):
    lines = []
    def _replace(m):
        mod,obj = m.groups()
        return f"from {_relimport2name(mod, fname)} import {obj}"
    for line in code.split('\n'):
        line = re.sub('__'+'file__', '_'+'file_', line) #Need to break _file_ or that line will be treated
        lines.append(_re_loc_import.sub(_replace,line))
    return '\n'.join(lines)

In [ ]:
#hide
code = "from .core import *\nnothing to see\nfrom .vision import bla1, bla2"
assert _deal_loc_import(code, 'local/data.py') == "from local.core import *\nnothing to see\nfrom local.vision import bla1, bla2"

In [ ]:
np.all([True,True])

True

In [ ]:
#export
def _script2notebook(fname, dic, silent=False):
    "Put the content of `fname` back in the notebooks it came from."
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    fname = Path(fname)
    with open(fname) as f: code = f.read()
    splits = _split(code)
    assert len(splits) == len(dic[fname]), f"Exported file from notebooks should have {len(dic[fname])} cells but has {len(splits)}."
    assert np.all([c1[0]==c2[1]] for c1,c2 in zip(splits, dic[fname]))
    splits = [(c2[0],c1[0],c1[1]) for c1,c2 in zip(splits, dic[fname])]
    nb_fnames = {s[1] for s in splits}
    for nb_fname in nb_fnames:
        nb = read_nb(nb_fname)
        for i,f,c in splits:
            c = _deal_loc_import(c, str(fname))
            if f == nb_fname:
                l = nb['cells'][i]['source'].split('\n')[0]
                nb['cells'][i]['source'] = l + '\n' + c
        NotebookNotary().sign(nb)
        nbformat.write(nb, nb_fname, version=4)
    if not silent: print(f"Converted {fname}.")

In [ ]:
if (Path.cwd()/'lib.pkl').exists(): os.remove(Path.cwd()/'lib.pkl')
notebook2script(all_fs=True, silent=True, to_pkl=True)
dic = pickle.load(open(Path.cwd()/'lib.pkl', 'rb'))

In [ ]:
_script2notebook('local/tabular/core.py', dic)

Converted local/tabular/core.py.


In [ ]:
#export
_manual_mods = ['__init__.py', 'imports.py', 'torch_imports.py', 'all.py', 'torch_basics.py', 'fp16_utils.py']

In [ ]:
#export
def script2notebook(folder='local', silent=False):
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    for f in Path(folder).glob('**/*.py'):
        if f.name not in _manual_mods: _script2notebook(f, silent=silent)

In [ ]:
script2notebook()

Converted local/test.py.
Converted local/metrics.py.
Converted local/optimizer.py.
Converted local/core.py.
Converted local/layers.py.
Converted local/script.py.
Converted local/torch_core.py.
Converted local/learner.py.
Converted local/tabular/core.py.
Converted local/tabular/model.py.
Converted local/callback/fp16.py.
Converted local/callback/tracker.py.
Converted local/callback/schedule.py.
Converted local/callback/mixup.py.
Converted local/callback/rnn.py.
Converted local/callback/progress.py.
Converted local/callback/hook.py.
Converted local/vision/augment.py.
Converted local/vision/core.py.
Converted local/vision/models/xresnet.py.
Converted local/utils/test.py.
Converted local/text/core.py.
Converted local/text/data.py.
Converted local/text/models/awdlstm.py.
Converted local/text/models/core.py.
Converted local/text/models/qrnn.py.
Converted local/notebook/showdoc.py.
Converted local/notebook/core.py.
Converted local/notebook/export.py.
Converted local/notebook/export2html.py.
C

## Diff notebook - library

In [ ]:
#export
def _print_diff_py(code1, code2, fname):
    split1,split2 = _split(code1),_split(code2)
    diff = []
    for (i1,f1,c1),(i2,f2,c2) in zip_longest(split1, split2):
        if c1 != c2: diff.append(f'Cell {i1} in {f1}:\n{c1}\n\nExported code from cell {i2} of {f2}:\n{c2}')
    if len(diff) > 0: 
        print(f"Diff notebook and script in {fname}")
        print("\n\n\n".join(diff))

In [ ]:
#export
def _print_diff_txt(code1, code2, fname):
    split1,split2 = code1.split("\n"),code2.split("\n")
    diff = []
    for t1,t2 in zip(split1, split2):
        if t1 is None: diff.append(f'Exported code cell {t2[0]} in {t2[1]}:\n{c1}\n\nExported code from cell {i2} of {f2}:\n{c2}')
        if c1 != c2: diff.append(f'Cell {i1} in {f1}:\n{c1}\n\nExported code from cell {i2} of {f2}:\n{c2}')
    if len(diff) > 0: 
        print(f"Diff notebook and script in {fname}")
        print("\n\n\n".join(diff))

In [ ]:
import subprocess

In [ ]:
#export
def _print_diff(code1, code2, fname):
    diff = difflib.ndiff(code1, code2)
    sys.stdout.writelines(diff)
    #for l in difflib.context_diff(code1, code2): print(l)
    #_print_diff_py(code1, code2, fname) if fname.endswith('.py') else _print_diff_txt(code1, code2, fname)

In [ ]:
#export
def diff_nb_script(lib_folder='local'):
    tmp_path1,tmp_path2 = Path.cwd()/'tmp_lib',Path.cwd()/'tmp_lib1'
    shutil.copytree(Path.cwd()/lib_folder, tmp_path1)
    try:
        notebook2script(all_fs=True, silent=True)
        shutil.copytree(Path.cwd()/lib_folder, tmp_path2)
        shutil.rmtree(Path.cwd()/lib_folder)
        shutil.copytree(tmp_path1, Path.cwd()/lib_folder)
        res = subprocess.run(['diff', '-ru', 'tmp_lib1', lib_folder], stdout=subprocess.PIPE)
        print(res.stdout.decode('utf-8'))
    finally: 
        shutil.rmtree(tmp_path1)
        shutil.rmtree(tmp_path2)

In [ ]:
diff_nb_script()

## Export

In [ ]:
#hide
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_torch_core.ipynb.
Converted 01b_script.ipynb.
Converted 01c_dataloader.ipynb.
Converted 02_data_transforms.ipynb.
Converted 03_data_pipeline.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 11a_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_test_models_core.ipynb.
Converted 34_callback_rnn.ipynb.
Converted 35_tutorial_wikitext.ipynb.
Conve